In [11]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit
import re

spark = SparkSession.builder.getOrCreate()

# listamos todas las tablas en lh_bronze
all_tables = spark.catalog.listTables("lh_bronze")

# filtramos con regex: brz_open_meteo_*_mensual
pattern = r"^brz_open_meteo_.*_mensual$"
matched_tables = [t.name for t in all_tables if re.match(pattern, t.name)]

# mapeo regex -> geo_name
geo_map = {
    "andalucia": "Andalucía",
    "aragon": "Aragón",
    "asturias": "Principado de Asturias",
    "baleares": "Islas Baleares",
    "canarias": "Islas Canarias",
    "cantabria": "Cantabria",
    "castilla_la_mancha": "Castilla-La Mancha",
    "castilla_leon": "Castilla y León",
    "cataluna": "Cataluña",
    "ceuta": "Comunidad de Ceuta",
    "extremadura": "Extremadura",
    "galicia": "Galicia",
    "madrid": "Comunidad de Madrid",
    "melilla": "Comunidad de Melilla",
    "murcia": "Región de Murcia",
    "navarra": "Comunidad de Navarra",
    "pais_vasco": "País Vasco",
    "peninsula": "Península",
    "rioja": "La Rioja",
    "valencia": "Comunidad Valenciana"
}

# listamos límites geográficos
geo_limits = ["peninsular", "canarias", "baleares", "ceuta", "melilla"]

# función auxiliar para extraer geo_name desde nombre de tabla
def extract_geo_name_from_table_name(table_name):
    for key in geo_map:
        if key in table_name.lower():
            return geo_map[key]
    return "Desconocido"

# función auxiliar para determinar geo_limit
def extract_geo_limit_from_table_name(table_name):
    name_lower = table_name.lower()
    for limit in geo_limits:
        if limit in name_lower:
            return limit
    # Si no se encuentra, asumimos "peninsular" por defecto
    return "peninsular"

# leer, agregar geo_name y geo_limit, luego unir todo
df_union = None

for table in matched_tables:
    df = spark.read.table(f"lh_bronze.{table}")
    geo_name = extract_geo_name_from_table_name(table)
    geo_limit = extract_geo_limit_from_table_name(table)
    
    df = df.withColumn("geo_name", lit(geo_name)) \
           .withColumn("geo_limit", lit(geo_limit))
    
    if df_union is None:
        df_union = df
    else:
        df_union = df_union.unionByName(df)

StatementMeta(, 018b44f4-ac57-4c77-a25a-5f4a2cff19cb, 13, Finished, Available, Finished)

In [12]:
# un display inicial
display(df_union.limit(5))
print(f"Total de filas iniciales: {df_union.count():,}")

StatementMeta(, 018b44f4-ac57-4c77-a25a-5f4a2cff19cb, 14, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, a77c7c41-970c-4a12-910c-76be3329228a)

Total de filas iniciales: 684


In [13]:
print(df_union.columns)

StatementMeta(, 018b44f4-ac57-4c77-a25a-5f4a2cff19cb, 15, Finished, Available, Finished)

['region', 'year', 'month', 'temperature_2m_max', 'temperature_2m_min', 'temperature_2m_mean', 'precipitation_sum', 'wind_speed_10m_max', 'ingestion_date', 'geo_name', 'geo_limit']


In [14]:
from pyspark.sql import functions as F

# eliminar columnas que tienen todas en nulo
# Contar cuántos valores no nulos tiene cada columna
non_null_counts = df_union.select([
    F.count(F.col(c)).alias(c)
    for c in df_union.columns
]).collect()[0].asDict()

# Filtrar las columnas que tienen al menos un valor no nulo
cols_to_keep = [c for c, count in non_null_counts.items() if count > 0]

# Crear nuevo DataFrame sin las columnas completamente nulas
df_no_null_cols = df_union.select(cols_to_keep)

print(f"Columnas eliminadas: {[c for c in df_union.columns if c not in cols_to_keep]}")
print(f"Total de columnas finales: {len(df_no_null_cols.columns)}")

# Mostrar el resultado
display(df_no_null_cols.limit(5))

StatementMeta(, 018b44f4-ac57-4c77-a25a-5f4a2cff19cb, 16, Finished, Available, Finished)

Columnas eliminadas: []
Total de columnas finales: 11


SynapseWidget(Synapse.DataFrame, 108bfc43-5df6-48b8-aabb-302c1a90208a)

In [15]:
# eliminamos duplicados según todas las columnas excepto ingestion_timestamp
df_clean = df_no_null_cols.dropDuplicates(['region', 'year', 'month', 'temperature_2m_max', 'temperature_2m_min', 'temperature_2m_mean', 'precipitation_sum', 'wind_speed_10m_max','geo_name', 'geo_limit'])

display(df_clean.limit(5))
print(f"Total después de limpiar duplicados: {df_clean.count():,}")

StatementMeta(, 018b44f4-ac57-4c77-a25a-5f4a2cff19cb, 17, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 315755c7-2a81-4157-b3c7-24312a1ae615)

Total después de limpiar duplicados: 684


In [16]:
# imprimimos valores únicos de cada columna
cols = ['region', 'year', 'month','geo_name', 'geo_limit']

for c in cols:
    uniques = [row[c] for row in df_clean.select(c).distinct().collect()]
    count_uniques = df_clean.select(c).distinct().count()
    print(f"\n📌 Valores únicos en '{c}': {count_uniques}")
    print(uniques)

StatementMeta(, 018b44f4-ac57-4c77-a25a-5f4a2cff19cb, 18, Finished, Available, Finished)


📌 Valores únicos en 'region': 57
['brz_andalucia_2023_mensual', 'brz_aragon_2023_mensual', 'brz_asturias_2023_mensual', 'brz_baleares_2023_mensual', 'brz_canarias_2023_mensual', 'brz_cantabria_2023_mensual', 'brz_castilla_la_mancha_2023_mensual', 'brz_castilla_leon_2023_mensual', 'brz_cataluna_2023_mensual', 'brz_ceuta_2023_mensual', 'brz_extremadura_2023_mensual', 'brz_galicia_2023_mensual', 'brz_madrid_2023_mensual', 'brz_melilla_2023_mensual', 'brz_murcia_2023_mensual', 'brz_navarra_2023_mensual', 'brz_pais_vasco_2023_mensual', 'brz_rioja_2023_mensual', 'brz_valencia_2023_mensual', 'brz_andalucia_2024_mensual', 'brz_aragon_2024_mensual', 'brz_asturias_2024_mensual', 'brz_baleares_2024_mensual', 'brz_canarias_2024_mensual', 'brz_cantabria_2024_mensual', 'brz_castilla_la_mancha_2024_mensual', 'brz_castilla_leon_2024_mensual', 'brz_cataluna_2024_mensual', 'brz_ceuta_2024_mensual', 'brz_extremadura_2024_mensual', 'brz_galicia_2024_mensual', 'brz_madrid_2024_mensual', 'brz_melilla_2024_

In [17]:
# eliminamos las columnas que creamos no necesarios y/o que solo tengan 1 valor único
df_clean = df_clean.drop("region", "ingestion_date", )

display(df_clean.limit(2))

StatementMeta(, 018b44f4-ac57-4c77-a25a-5f4a2cff19cb, 19, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 6a0183ea-59c0-488b-8cfb-5ffe4dbfb4c1)

In [18]:
# renombramos las columnas para mejor comprensión
from pyspark.sql.functions import col

df_clean = (
    df_clean
    .withColumnRenamed("temperature_2m_max", "max_temp")
    .withColumnRenamed("temperature_2m_min", "min_temp")
    .withColumnRenamed("temperature_2m_mean", "mean_temp")
    .withColumnRenamed("wind_speed_10m_max", "wind_speed")
)

display(df_clean.limit(2))

StatementMeta(, 018b44f4-ac57-4c77-a25a-5f4a2cff19cb, 20, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 041895e3-aa07-47c8-b2ab-468d06f88793)

In [19]:
# guardamos en capa Silver
df_clean.write.mode("overwrite").saveAsTable("lh_silver.slv_open_meteo_monthly_cleaned")
print("✅ La tabla lh_silver.slv_open_meteo_monthly_cleaned se ha creado correctamente tras la limpieza.")

StatementMeta(, 018b44f4-ac57-4c77-a25a-5f4a2cff19cb, 21, Finished, Available, Finished)

✅ La tabla lh_silver.slv_open_meteo_monthly_cleaned se ha creado correctamente tras la limpieza.
